In [1]:
import requests
import json
import csv
import pandas as pd
from pandas.io.json import json_normalize

z = pd.read_csv(
    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\Sourcing\\pconf.csv",
    low_memory=False,
    index_col=False,
    encoding="unicode_escape",
).fillna("")
s = pd.read_csv(
    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\ama.csv",
    low_memory=False,
    index_col=False,
    encoding="unicode_escape",
).fillna("")
# acct = pd.read_csv(
#    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\acct_npi.csv",
#    low_memory=False,
#    index_col=False,
#    encoding="unicode_escape",
# ).fillna("")
cross = pd.read_csv(
    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\cross_npi.csv",
    low_memory=False,
    index_col=False,
    encoding="unicode_escape",
    dtype="object",
).fillna("")


cross = cross[cross["NPI"] != ""]
# acct1 = acct[acct["Person Account: NPI Number"] != ""]
s.rename(columns={"AMA Specialty: AMA Specialty Name": "Specialization"})
s = s.drop_duplicates()
s1 = s[s["Taxonomy Code"] != ""]


errors = []
data = []

# merge = pd.concat([a,b,c], ignore_index=True)
# file1 = merge[(merge['Mail Name'] != '© 2023 by the American Medical Association') & merge['Mail Name'] != 'Â© 2023 by the American Medical Association']
# file1 = file1[['Mail  Last Name', 'Mail First Name', 'MD-DO Flag', 'City', 'State', 'Zip', 'Primary Specialty']]
# file = file1.rename(
#    columns = {'Mail  Last Name': 'last_name',
#            'Mail First Name': 'first_name',
#            'Mail City': 'City',
#            'State': 'state',
#            'Zip': 'Postal Code',
#            'MD-DO Flag': 'Practitioner Type'
#            })
data_dfs = pd.DataFrame(
    columns=[
        "number",
        "First Name",
        "Last Name",
        "Gender",
        "basic.enumeration_date",
        "country_name",
        "address_purpose",
        "address_1",
        "city",
        "state_x",
        "postal_code",
        "telephone_number",
        "code",
        "taxonomy_group",
        "desc",
        "state_y",
        "license",
        "primary",
    ]
).fillna("")

data_df = pd.DataFrame(
    columns=[
        "number",
        "First Name",
        "Last Name",
        "Gender",
        "basic.enumeration_date",
        "country_name",
        "address_purpose",
        "address_1",
        "city",
        "state_x",
        "postal_code",
        "telephone_number",
        "code",
        "taxonomy_group",
        "desc",
        "state_y",
        "license",
        "primary",
    ]
).fillna("")

url = "https://npiregistry.cms.hhs.gov/api/?version=2.1"


file = z.fillna("")

file_df = file[["First Name", "Last Name"]]

for row in file_df.index:
    first_name, last_name = (
        file_df["First Name"][row],
        file_df["Last Name"][row],
    )
    # file_df["Last Name"].str[:2][row]
    parameters = {
        "version": "2.1",
        "first_name": first_name,
        "last_name": last_name,
        "pretty": "true",
    }

    # raises exception when not a 2xx response

    response = requests.get(url, params=parameters).json()
    # response.raise_for_status()
    #
    # if response.status_code != 204:
    #    return response.json()
    for i in response:
        if i == "result_count":
            if response["result_count"] < 1:
                errors.append(parameters)
            elif response["result_count"] >= 1:
                data.append(response)
        else:
            errors.append(parameters)
    for l in data:
        data1 = pd.json_normalize(
            l["results"], "addresses", ["number", "enumeration_type"]
        )
        data1 = data1.drop_duplicates(subset="number")
        data2 = pd.json_normalize(l["results"], "taxonomies", ["number"])
        data4 = pd.json_normalize(l["results"])
        file1 = pd.merge_ordered(data1, data2, on="number", how="outer")
        file2 = pd.merge_ordered(file1, data4, on="number", how="outer")

        file3 = file2[
            [
                "number",
                "basic.first_name",
                "basic.last_name",
                "basic.gender",
                "basic.enumeration_date",
                "country_name",
                "address_purpose",
                "address_1",
                "city",
                "state_x",
                "postal_code",
                "telephone_number",
                "code",
                "taxonomy_group",
                "desc",
                "state_y",
                "license",
                "primary",
            ]
        ]
        file4a = file3.rename(
            columns={
                "basic.first_name": "First Name",
                "basic.last_name": "Last Name",
                "basic.middle_name": "Middle Name",
                "basic.credential": "Practitioner Type",
                "basic.gender": "Gender",
            }
        ).drop_duplicates(subset="number")

        file4b = pd.merge_ordered(
            file4a, cross, left_on="number", right_on="NPI", how="left"
        ).fillna("")
        file4c = pd.merge_ordered(
            file4b, s1, left_on="code", right_on="Taxonomy Code", how="left"
        ).fillna("")
        file4 = file4c[file4c["AMA Specialty: AMA Specialty Name"] != ""]

        if len(data_df) < 500:
            data_df = pd.concat([data_df, file4]).drop_duplicates(subset="number")
        elif len(data_df) >= 500:
            data_df.to_csv("e2.csv", index=False)
            data_df1 = pd.concat([data_df1, file4])
            data_df1 = data_df1.drop_duplicates(subset="number")
        else:
            data_df1.to_csv("e3.csv", index=False)
            break

In [1]:
#########################1
import pandas as pd

ama = pd.read_csv(
    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\ama.csv",
    low_memory=False,
    index_col=False,
    encoding="unicode_escape",
    dtype=object,
).fillna("")

df = pd.read_csv(
    "C:\\Users\\mihma\\OneDrive\\Desktop\\Data Work\\sand.csv",
    usecols=[
        "NPI",
        "Entity Type Code",
        "Provider Organization Name (Legal Business Name)",
        "Provider Last Name (Legal Name)",
        "Provider First Name",
        "Provider Middle Name",
        "Provider Credential Text",
        "Provider First Line Business Mailing Address",
        "Provider Second Line Business Mailing Address",
        "Provider Business Practice Location Address City Name",
        "Provider Business Practice Location Address State Name",
        "Provider Business Practice Location Address Postal Code",
        "Provider Business Practice Location Address Country Code (If outside U.S.)",
        "Provider Business Practice Location Address Telephone Number",
        "Provider Business Practice Location Address Fax Number",
        "Provider Enumeration Date",
        "NPI Deactivation Date",
        "NPI Reactivation Date",
        "Provider Gender Code",
        "Authorized Official Last Name",
        "Authorized Official First Name",
        "Authorized Official Middle Name",
        "Authorized Official Title or Position",
        "Authorized Official Telephone Number",
        "Healthcare Provider Taxonomy Code_1",
        "Provider License Number_1",
        "Provider License Number State Code_1",
        "Healthcare Provider Primary Taxonomy Switch_1",
        "Healthcare Provider Taxonomy Code_2",
        "Provider License Number_2",
        "Provider License Number State Code_2",
        "Healthcare Provider Primary Taxonomy Switch_2",
        "Healthcare Provider Taxonomy Code_3",
        "Provider License Number_3",
        "Provider License Number State Code_3",
        "Healthcare Provider Primary Taxonomy Switch_3",
        "Healthcare Provider Taxonomy Code_4",
        "Provider License Number_4",
        "Provider License Number State Code_4",
        "Healthcare Provider Primary Taxonomy Switch_4",
        "Healthcare Provider Taxonomy Code_5",
        "Provider License Number_5",
        "Provider License Number State Code_5",
        "Healthcare Provider Primary Taxonomy Switch_5",
        "Healthcare Provider Taxonomy Code_6",
        "Provider License Number_6",
        "Provider License Number State Code_6",
        "Healthcare Provider Primary Taxonomy Switch_6",
        "Healthcare Provider Taxonomy Code_7",
        "Provider License Number_7",
        "Provider License Number State Code_7",
        "Healthcare Provider Primary Taxonomy Switch_7",
        "Healthcare Provider Taxonomy Code_8",
        "Provider License Number_8",
        "Provider License Number State Code_8",
        "Healthcare Provider Primary Taxonomy Switch_8",
        "Healthcare Provider Taxonomy Code_9",
        "Provider License Number_9",
        "Provider License Number State Code_9",
        "Healthcare Provider Primary Taxonomy Switch_9",
        "Healthcare Provider Taxonomy Code_10",
        "Provider License Number_10",
        "Provider License Number State Code_10",
        "Healthcare Provider Primary Taxonomy Switch_10",
        "Healthcare Provider Taxonomy Code_11",
        "Provider License Number_11",
        "Provider License Number State Code_11",
        "Healthcare Provider Primary Taxonomy Switch_11",
        "Healthcare Provider Taxonomy Code_12",
        "Provider License Number_12",
        "Provider License Number State Code_12",
        "Healthcare Provider Primary Taxonomy Switch_12",
        "Healthcare Provider Taxonomy Code_13",
        "Provider License Number_13",
        "Provider License Number State Code_13",
        "Healthcare Provider Primary Taxonomy Switch_13",
        "Healthcare Provider Taxonomy Code_14",
        "Provider License Number_14",
        "Provider License Number State Code_14",
        "Healthcare Provider Primary Taxonomy Switch_14",
        "Healthcare Provider Taxonomy Code_15",
        "Provider License Number_15",
        "Provider License Number State Code_15",
        "Healthcare Provider Primary Taxonomy Switch_15",
    ],
    chunksize=100000,
    low_memory=False,
    index_col=False,
    encoding="unicode_escape",
    dtype=object,
)  # .fillna("")
for ch in df:
    sand = pd.concat(df).fillna("")
t2 = sand[sand["Entity Type Code"] == "2"]
p1 = sand[sand["Entity Type Code"] == "1"]
s = sand[
    [
        "NPI",
        "Healthcare Provider Taxonomy Code_1",
        "Provider License Number_1",
        "Provider License Number State Code_1",
        "Healthcare Provider Primary Taxonomy Switch_1",
        "Healthcare Provider Taxonomy Code_2",
        "Provider License Number_2",
        "Provider License Number State Code_2",
        "Healthcare Provider Primary Taxonomy Switch_2",
        "Healthcare Provider Taxonomy Code_3",
        "Provider License Number_3",
        "Provider License Number State Code_3",
        "Healthcare Provider Primary Taxonomy Switch_3",
        "Healthcare Provider Taxonomy Code_4",
        "Provider License Number_4",
        "Provider License Number State Code_4",
        "Healthcare Provider Primary Taxonomy Switch_4",
        "Healthcare Provider Taxonomy Code_5",
        "Provider License Number_5",
        "Provider License Number State Code_5",
        "Healthcare Provider Primary Taxonomy Switch_5",
        "Healthcare Provider Taxonomy Code_6",
        "Provider License Number_6",
        "Provider License Number State Code_6",
        "Healthcare Provider Primary Taxonomy Switch_6",
        "Healthcare Provider Taxonomy Code_7",
        "Provider License Number_7",
        "Provider License Number State Code_7",
        "Healthcare Provider Primary Taxonomy Switch_7",
        "Healthcare Provider Taxonomy Code_8",
        "Provider License Number_8",
        "Provider License Number State Code_8",
        "Healthcare Provider Primary Taxonomy Switch_8",
        "Healthcare Provider Taxonomy Code_9",
        "Provider License Number_9",
        "Provider License Number State Code_9",
        "Healthcare Provider Primary Taxonomy Switch_9",
        "Healthcare Provider Taxonomy Code_10",
        "Provider License Number_10",
        "Provider License Number State Code_10",
        "Healthcare Provider Primary Taxonomy Switch_10",
        "Healthcare Provider Taxonomy Code_11",
        "Provider License Number_11",
        "Provider License Number State Code_11",
        "Healthcare Provider Primary Taxonomy Switch_11",
        "Healthcare Provider Taxonomy Code_12",
        "Provider License Number_12",
        "Provider License Number State Code_12",
        "Healthcare Provider Primary Taxonomy Switch_12",
        "Healthcare Provider Taxonomy Code_13",
        "Provider License Number_13",
        "Provider License Number State Code_13",
        "Healthcare Provider Primary Taxonomy Switch_13",
        "Healthcare Provider Taxonomy Code_14",
        "Provider License Number_14",
        "Provider License Number State Code_14",
        "Healthcare Provider Primary Taxonomy Switch_14",
        "Healthcare Provider Taxonomy Code_15",
        "Provider License Number_15",
        "Provider License Number State Code_15",
        "Healthcare Provider Primary Taxonomy Switch_15",
    ]
]
p1 = p1[
    [
        "NPI",
        "Entity Type Code",
        "Provider Organization Name (Legal Business Name)",
        "Provider Last Name (Legal Name)",
        "Provider First Name",
        "Provider Middle Name",
        "Provider Credential Text",
        "Provider First Line Business Mailing Address",
        "Provider Second Line Business Mailing Address",
        "Provider Business Practice Location Address City Name",
        "Provider Business Practice Location Address State Name",
        "Provider Business Practice Location Address Postal Code",
        "Provider Business Practice Location Address Country Code (If outside U.S.)",
        "Provider Business Practice Location Address Telephone Number",
        "Provider Business Practice Location Address Fax Number",
        "Provider Enumeration Date",
        "NPI Deactivation Date",
        "NPI Reactivation Date",
        "Provider Gender Code",
        "Provider License Number State Code_1",
        "Provider License Number State Code_2",
        "Provider License Number State Code_3",
        "Provider License Number State Code_4",
        "Provider License Number State Code_5",
        "Provider License Number State Code_6",
        "Provider License Number State Code_7",
        "Provider License Number State Code_8",
        "Provider License Number State Code_9",
        "Provider License Number State Code_10",
        "Provider License Number State Code_11",
        "Provider License Number State Code_12",
        "Provider License Number State Code_13",
        "Provider License Number State Code_14",
        "Provider License Number State Code_15",
    ]
].fillna("")

p1["Gender"] = p1["Provider Gender Code"].replace({"F": "Female", "M": "Male"})

p1["License (Active)"] = (
    p1[
        [
            "Provider License Number State Code_1",
            "Provider License Number State Code_2",
            "Provider License Number State Code_3",
            "Provider License Number State Code_4",
            "Provider License Number State Code_5",
            "Provider License Number State Code_6",
            "Provider License Number State Code_7",
            "Provider License Number State Code_8",
            "Provider License Number State Code_9",
            "Provider License Number State Code_10",
            "Provider License Number State Code_11",
            "Provider License Number State Code_12",
            "Provider License Number State Code_13",
            "Provider License Number State Code_14",
            "Provider License Number State Code_15",
        ]
    ]
    .astype("str")
    .agg(";".join, axis=1)
    .drop_duplicates()
    .fillna("")
)
p1["License (Active)"] = (
    p1["License (Active)"]
    .replace(
        {
            ";;;;;;;;;;;;;;$": "",
            ";;;;;;;;;;;;;$": "",
            ";;;;;;;;;;;;$": "",
            ";;;;;;;;;;;$": "",
            ";;;;;;;;;;$": "",
            ";;;;;;;;;$": "",
            ";;;;;;;;$": "",
            ";;;;;;;$": "",
            ";;;;;;$": "",
            ";;;;;$": "",
            ";;;;$": "",
            ";;;$": "",
            ";;$": "",
            ";$": "",
            "^;;;;;;;;;;;;;;": "",
            "^;;;;;;;;;;;;;": "",
            "^;;;;;;;;;;;;$": "",
            "^;;;;;;;;;;;": "",
            "^;;;;;;;;;;": "",
            "^;;;;;;;;;": "",
            "^;;;;;;;;": "",
            "^;;;;;;;": "",
            "^;;;;;;": "",
            "^;;;;;": "",
            "^;;;;": "",
            "^;;;": "",
            "^;;": "",
            "^;": "",
        },
        regex=True,
    )
    .fillna("")
)
p2 = p1[
    [
        "NPI",
        "Gender",
        "Entity Type Code",
        "Provider Organization Name (Legal Business Name)",
        "Provider Last Name (Legal Name)",
        "Provider First Name",
        "Provider Middle Name",
        "Provider Credential Text",
        "Provider First Line Business Mailing Address",
        "Provider Second Line Business Mailing Address",
        "Provider Business Practice Location Address City Name",
        "Provider Business Practice Location Address State Name",
        "Provider Business Practice Location Address Postal Code",
        "Provider Business Practice Location Address Country Code (If outside U.S.)",
        "Provider Business Practice Location Address Telephone Number",
        "Provider Business Practice Location Address Fax Number",
        "Provider Enumeration Date",
        "NPI Deactivation Date",
        "NPI Reactivation Date",
        "License (Active)",
        "Provider Gender Code",
    ]
]
t3 = t2[
    [
        "NPI",
        "Entity Type Code",
        "Provider Credential Text",
        "Provider First Line Business Mailing Address",
        "Provider Second Line Business Mailing Address",
        "Provider Business Practice Location Address City Name",
        "Provider Business Practice Location Address State Name",
        "Provider Business Practice Location Address Postal Code",
        "Provider Business Practice Location Address Country Code (If outside U.S.)",
        "Provider Business Practice Location Address Telephone Number",
        "Provider Enumeration Date",
        "Authorized Official Last Name",
        "Authorized Official First Name",
        "Authorized Official Middle Name",
        "Authorized Official Title or Position",
        "Authorized Official Telephone Number",
    ]
]

s1 = s[
    [
        "NPI",
        "Healthcare Provider Primary Taxonomy Switch_1",
        "Healthcare Provider Taxonomy Code_1",
    ]
].rename(
    columns={
        "Healthcare Provider Taxonomy Code_1": "code",
        "Healthcare Provider Primary Taxonomy Switch_1": "switch",
    }
)
s2 = s[
    [
        "NPI",
        "Healthcare Provider Primary Taxonomy Switch_2",
        "Healthcare Provider Taxonomy Code_2",
    ]
].rename(
    columns={
        "Healthcare Provider Taxonomy Code_2": "code",
        "Healthcare Provider Primary Taxonomy Switch_2": "switch",
    }
)
s3 = s[
    [
        "NPI",
        "Healthcare Provider Primary Taxonomy Switch_3",
        "Healthcare Provider Taxonomy Code_3",
    ]
].rename(
    columns={
        "Healthcare Provider Taxonomy Code_3": "code",
        "Healthcare Provider Primary Taxonomy Switch_3": "switch",
    }
)
s4 = s[
    [
        "NPI",
        "Healthcare Provider Primary Taxonomy Switch_4",
        "Healthcare Provider Taxonomy Code_4",
    ]
].rename(
    columns={
        "Healthcare Provider Taxonomy Code_4": "code",
        "Healthcare Provider Primary Taxonomy Switch_4": "switch",
    }
)
s5 = s[
    [
        "NPI",
        "Healthcare Provider Primary Taxonomy Switch_5",
        "Healthcare Provider Taxonomy Code_5",
    ]
].rename(
    columns={
        "Healthcare Provider Taxonomy Code_5": "code",
        "Healthcare Provider Primary Taxonomy Switch_5": "switch",
    }
)
s6 = s[
    [
        "NPI",
        "Healthcare Provider Primary Taxonomy Switch_6",
        "Healthcare Provider Taxonomy Code_6",
    ]
].rename(
    columns={
        "Healthcare Provider Taxonomy Code_6": "code",
        "Healthcare Provider Primary Taxonomy Switch_6": "switch",
    }
)
s7 = s[
    [
        "NPI",
        "Healthcare Provider Primary Taxonomy Switch_7",
        "Healthcare Provider Taxonomy Code_7",
    ]
].rename(
    columns={
        "Healthcare Provider Taxonomy Code_7": "code",
        "Healthcare Provider Primary Taxonomy Switch_7": "switch",
    }
)
s8 = s[
    [
        "NPI",
        "Healthcare Provider Primary Taxonomy Switch_8",
        "Healthcare Provider Taxonomy Code_8",
    ]
].rename(
    columns={
        "Healthcare Provider Taxonomy Code_8": "code",
        "Healthcare Provider Primary Taxonomy Switch_8": "switch",
    }
)
s9 = s[
    [
        "NPI",
        "Healthcare Provider Primary Taxonomy Switch_9",
        "Healthcare Provider Taxonomy Code_9",
    ]
].rename(
    columns={
        "Healthcare Provider Taxonomy Code_9": "code",
        "Healthcare Provider Primary Taxonomy Switch_9": "switch",
    }
)
s10 = s[
    [
        "NPI",
        "Healthcare Provider Primary Taxonomy Switch_10",
        "Healthcare Provider Taxonomy Code_10",
    ]
].rename(
    columns={
        "Healthcare Provider Taxonomy Code_10": "code",
        "Healthcare Provider Primary Taxonomy Switch_10": "switch",
    }
)
s11 = s[
    [
        "NPI",
        "Healthcare Provider Primary Taxonomy Switch_11",
        "Healthcare Provider Taxonomy Code_11",
    ]
].rename(
    columns={
        "Healthcare Provider Taxonomy Code_11": "code",
        "Healthcare Provider Primary Taxonomy Switch_11": "switch",
    }
)
s12 = s[
    [
        "NPI",
        "Healthcare Provider Primary Taxonomy Switch_12",
        "Healthcare Provider Taxonomy Code_12",
    ]
].rename(
    columns={
        "Healthcare Provider Taxonomy Code_12": "code",
        "Healthcare Provider Primary Taxonomy Switch_12": "switch",
    }
)
s13 = s[
    [
        "NPI",
        "Healthcare Provider Primary Taxonomy Switch_13",
        "Healthcare Provider Taxonomy Code_13",
    ]
].rename(
    columns={
        "Healthcare Provider Taxonomy Code_13": "code",
        "Healthcare Provider Primary Taxonomy Switch_13": "switch",
    }
)
s14 = s[
    [
        "NPI",
        "Healthcare Provider Primary Taxonomy Switch_14",
        "Healthcare Provider Taxonomy Code_14",
    ]
].rename(
    columns={
        "Healthcare Provider Taxonomy Code_14": "code",
        "Healthcare Provider Primary Taxonomy Switch_14": "switch",
    }
)
s15 = s[
    [
        "NPI",
        "Healthcare Provider Primary Taxonomy Switch_15",
        "Healthcare Provider Taxonomy Code_15",
    ]
].rename(
    columns={
        "Healthcare Provider Taxonomy Code_15": "code",
        "Healthcare Provider Primary Taxonomy Switch_15": "switch",
    }
)


s1 = s1[s1["switch"] == "Y"]
s2 = s2[s2["switch"] == "Y"]
s3 = s3[s3["switch"] == "Y"]
s4 = s4[s4["switch"] == "Y"]
s5 = s5[s5["switch"] == "Y"]
s6 = s6[s6["switch"] == "Y"]
s7 = s7[s7["switch"] == "Y"]
s8 = s8[s8["switch"] == "Y"]
s9 = s9[s9["switch"] == "Y"]
s10 = s10[s10["switch"] == "Y"]
s11 = s11[s11["switch"] == "Y"]
s12 = s12[s12["switch"] == "Y"]
s13 = s13[s13["switch"] == "Y"]
s14 = s14[s14["switch"] == "Y"]
s15 = s15[s15["switch"] == "Y"]

##############2
sss = pd.concat([s1, s2, s3, s4, s5, s6, s7, s8, s9, s10, s11, s12, s13, s14, s15])

t4 = pd.merge_ordered(t3, sss, on="NPI", how="left")
p3 = pd.merge_ordered(p2, sss, on="NPI", how="left")


############1
t4.drop_duplicates(subset="NPI")
p3.drop_duplicates(subset="NPI")
t5 = pd.merge_ordered(
    t4, ama, left_on="code", right_on="Taxonomy Code", how="left"
).fillna("")
t5 = t5[t5["AMA Specialty: AMA Specialty Name"] != ""]
p4 = pd.merge_ordered(
    p3, ama, left_on="code", right_on="Taxonomy Code", how="left"
).fillna("")
p4 = p4[p4["AMA Specialty: AMA Specialty Name"] != ""]



p_cred = pd.Series(p5["Provider Credential Text"].values.flatten())
p_spec = pd.Series(p5["Classification"].values.flatten())
p5["PT"] = "MD"

p5["PT"].mask(p_cred.str.contains(r"DO", regex=True), "DO", inplace=True)
p5["PT"].mask(p_spec.str.contains(r"Physician Assistant"), "PA", inplace=True)
p5["PT"].mask(p_spec.str.contains(r"Nurse Practitioner"), "NP", inplace=True)
p5["PT"].mask(p_spec.str.contains(r"Podiatrist"), "DPM", inplace=True)
p5["PT"].mask(p_spec.str.contains(r"Nurse Anesthetist"), "CRNA", inplace=True)
p5["PT"].mask(p_cred.str.contains(r"MD|M.D"), "MD", inplace=True)
p5["Gender"] = p5["Gender"].replace({"F": "Female", "M": "Male"})
aq = pd.Series(p5["Acquisition Source"].values.flatten())

p5["Acquisition Source"].mask(
    p5["Acquisition Source"] == "", "Data Source", inplace=True
)



# up1.to_csv("920up1.csv", index=False)
# in1.to_csv("920in1.csv", index=False)
# acup1.to_csv("920acup1.csv", index=False)
# t5.to_csv("920t5.csv", index=False)

In [13]:
up1.to_csv("920up1.csv", index=False)
in1.to_csv("920in1.csv", index=False)
acup1.to_csv("920acup1.csv", index=False)
t5.to_csv("920t5.csv", index=False)

In [ ]:
import pandas as pd


acct = acct[acct["Person Account: Email"] != ""]
lede = lede[lede["Email"] != ""]
users = [
    "0056g000005Yehu",
    "0056g0000066eOF",
    "0056g0000066eOY",
    "0056g0000066eOZ",
    "0056g0000066eOn",
    "0056g000006LhyP",
    "0056g000006Mvn5",
    "0056g0000073I1s",
    "0055b00000Q6RCr",
]

# Distribute values based on colomn len
aaaa["User"] = (
    pd.np.tile(users, len(aaaa) // len(users)).tolist()
    + users[: len(aaaa) % len(users)]
)
za = aaaa.fillna("")

In [25]:
cross1 = lede[lede["NPI"] != ""].drop_duplicates(subset="NPI").fillna("")
confid1 = confid[confid["File - Email"] != ""].fillna("")
# acct = acct[acct['NPI'] != ''].drop_duplicates(subset='NPI')
# zb = pd.merge_ordered(za, s1, left_on='code', right_on='Code', how='left').fillna('').astype(object)
# z = zb[zb['Specialty: ID'] != '']
s1 = (
    pd.merge_ordered(za, confid, right_on="File - Email", left_on="Email", how="outer")
    .drop_duplicates(subset="NPI")
    .fillna("")
    .astype(object)
)
# atte = pd.merge_ordered(atte, user, left_on='User', right_on='Full Name', how='left')
# z1 = pd.merge_ordered(za, confid1, left_on='Email', right_on='File - Email', how='left').fillna('').astype(object)
s2 = (
    pd.merge_ordered(s1, cross1, left_on="NPI", right_on="NPI", how="outer")
    .drop_duplicates(subset="NPI")
    .fillna("")
    .astype(object)
)
# s4 = pd.merge_ordered(z1, acct, left_on='NPI', right_on='NPI', how='left').drop_duplicates(subset='NPI').fillna('').astype(object)
s5 = pd.merge_ordered(
    s1, lede, left_on="File - Email", right_on="Email", how="left"
).fillna("")

In [47]:
import pandas as pd

tax = pd.read_csv(
    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\final.csv",
    low_memory=False,
    index_col=False,
    encoding="unicode_escape",
    dtype="object",
).fillna("")
pst = pd.read_csv(
    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\spenam.csv",
    low_memory=False,
    index_col=False,
    encoding="unicode_escape",
    dtype="object",
).fillna("")

tax = tax.fillna("")
pst = pst.fillna("")

tax = tax[tax["Specialization"] != ""]
pst = pst[pst["Specialty: Specialty Name"] != ""]

z = pd.merge_ordered(
    pst,
    tax,
    left_on="Specialty: Specialty Name",
    right_on="Specialization",
    how="outer",
).fillna("")
# x = pd.merge_ordered(pst, tax, left_on='Residency/Fellowship Specialty', right_on='Specialization', how='left').fillna('')

# z = z[(z['Specialty ID'] != '') & (z['Code'] != '')]
# x = x[(x['Specialty ID'] != '') & (x['Code'] != '')]
#
# c = pd.concat([x,z])
# c = c.fillna('')

In [44]:
z.fillna("")
z[z["Code"] != ""].to_csv("secs.csv", index=False)

In [ ]:
import pandas as pd

df = pd.read_csv(
    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\june21.csv",
    low_memory=False,
    index_col=False,
    encoding="unicode_escape",
    dtype="object",
)
tax = pd.read_csv(
    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\TAX.csv",
    low_memory=False,
    index_col=False,
    encoding="unicode_escape",
    dtype="object",
)
pst = pd.read_csv(
    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\pst1.csv",
    low_memory=False,
    index_col=False,
    encoding="unicode_escape",
    dtype="object",
)
# pst1 = pd.merge_ordered(df, tax, on='Code', how='left')
pst1 = pst.fillna("")
pst2b = pd.merge_ordered(
    df, pst1, left_on="Specialization", right_on="Sub Specialty", how="left"
)
pst2b = pst2b.drop_duplicates(subset="number")
pb = pst2b.fillna("")
# pb = pst2b[pst2b['Specialty ID'] != '']
pst2a = pd.merge_ordered(
    df,
    pst1,
    left_on="Specialization",
    right_on="Residency/Fellowship Specialty",
    how="left",
)
pst2a = pst2a.drop_duplicates(subset="number")
pa = pst2a.fillna("")
# pa = pst2a[pst2a['Specialty ID'] != '']
pc = pd.concat([pa, pb])
pc_df = pc.drop_duplicates(subset="number").fillna("")
pc_df

In [47]:
pst2.to_csv("june212.csv", index=False)

In [2]:
import pandas as pd
import json
from pandas.io.json import json_normalize

file = input("Enter file name: ")

f = pd.read_json(
    f"C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\{file}.json"
)

In [63]:
# g = pd.read_csv("C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\19k.csv",low_memory=False,
# index_col=False, encoding='unicode_escape')
g = data_df.fillna("")
df = pd.read_csv(
    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\TAX.csv",
    low_memory=False,
    index_col=False,
    encoding="unicode_escape",
    dtype="object",
)
acct = pd.read_csv(
    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\acct_npi.csv",
    low_memory=False,
    index_col=False,
    encoding="unicode_escape",
)
cross = pd.read_csv(
    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\cross_npi.csv",
    low_memory=False,
    index_col=False,
    encoding="unicode_escape",
    dtype="object",
)
cross = cross.fillna("")
cross1 = cross[cross["NPI"] != ""]


# g = g.drop_duplicates(subset='number')
# g = g[(g['code'].isin(df['Code'])) & (~g['number'].isin(acct['NPI']))]
g = g.astype(object)
f1 = pd.merge_ordered(g, cross1, left_on="number", right_on="NPI", how="left")
f2 = pd.merge_ordered(g, acct, left_on="number", right_on="NPI", how="left")
f1 = f1.fillna("")
f2 = f2.fillna("")
f1a = f1[f1["Lead Record Id"] != ""]
f2a = f2[f2["Account Record Id"] != ""]
# f.to_csv('19k.csv', index=False)
# z = pd.concat([g,f])

In [ ]:
import pandas as pd

import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep


get_table(
    round,
    url="https://online.echoboards.org//nbessa//f?p=CRTSSA:17800:11269396482028::::::",
)
round_url = f"{url}/{round}"
page = requests.get(round_url)
soup = BeautifulSoup(page.text, "html.parser")

rows = []
for child in soup.find_all("table")[4].children:
    row = []
    for td in child:
        try:
            row.append(td.text.replace("\n", ""))
        except:
            continue
    if len(row) > 0:
        rows.append(row)
df = pd.DataFrame(rows[1:], columns=rows[0])
return df


for round in range(1, 39):
    table = get_table(round)
    table.to_csv(f"PL_table_matchweek_{round}.csv", index=False)
    sleep(np.random.randint(1, 10))

In [ ]:
import requests
from bs4 import BeautifulSoup

# URL of web page to be parsed
url = "https://online.echoboards.org/nbessa/f?p=CRTSSA:17800:11269396482028::::::"

# Make HTTP request to retrieve HTML source of page
response = requests.get(url)

# Parse HTML source using BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Find specific HTML element by tag, class, or ID
element = soup.find(
    "table",
    {
        "class": "t-Report-report",
    },
)

# Extract element's text or attribute
text = element.text
attribute = element["href"]
# Output results to console

print(text)
print(attribute)

In [ ]:
import csv

from salesforce_bulk import SalesforceBulk
from salesforce_bulk import CsvDictsAdapter

bulk = SalesforceBulk(username=username, password=password)

job = bulk.create_insert_job("user__c", contentType="CSV", concurrency="Parallel")

reader = csv.DictReader(open("user__c.csv"))
disbursals = []
for row in reader:
    disbursals.append(row)

csv_iter = CsvDictsAdapter(iter(disbursals))
batch = bulk.post_bulk_batch(job, csv_iter)
bulk.wait_for_batch(job, batch)
bulk.close_job(job)

print("Done. Data Uploaded.")

In [ ]:
bulk = SalesforceBulk(username=username, password=password)

job = bulk.create_insert_job(
    "custom_object__c", contentType="CSV", concurrency="Parallel"
)

disbursals = [dict(Name="custoom_object__c%d" % idx) for idx in xrange(5)]

csv_iter = CsvDictsAdapter(iter(disbursals))

batch = bulk.post_bulk_batch(job, csv_iter)

bulk.wait_for_batch(job, batch)

bulk.close_job(job)

print("Done. Data Uploaded.")

In [ ]:
import os
import pandas as pd

file_path = input("enter the path to the file you want to open")
for file in file_path:
    if file.endswith(".csv", "tsv"):
        df = pd.read_csv(file)
        print(df.head(100))
    elif file.endswith(".json"):
        df = pd.read_json(file)
        print(df.head(100))
    elif file.endswith(".xml"):
        df = pd.read_xml(file)
        print(df.head(100))
    elif file.endswith(".xls", "xlsx"):
        df = pd.read_excel(file)
        print(df.head(100))
    elif file.endswith(".hdf"):
        df = pd.read_hdf(file)
        print(df.head(100))
    elif file.endswith(".sql"):
        df = pd.read_sql(file)
        print(df.head(100))
    else:
        print("file format not supported")

In [ ]:
import os
import pandas as pd


def read_any(file):
    if file.endswith(".csv", "tsv"):
        df = pd.read_csv(file)
    elif file.endswith(".json"):
        df = pd.read_json(file)
    elif file.endswith(".xml"):
        df = pd.read_xml(file)
    elif file.endswith(".xls", "xlsx"):
        df = pd.read_excel(file)
    elif file.endswith(".hdf"):
        df = pd.read_hdf(file)
    elif file.endswith(".sql"):
        df = pd.read_sql(file)
    else:
        raise ValueError(f"Unsupported filetype: {file}")
    return df


if __name__ == "__main__":
    # here or wherever it is used
    file_path = input("enter the path to the file you want to open")
    df = read_any(file_path)
    print(df.head(100))

In [ ]:
import os.path

READER_MAP = {
    "xlsx": pd.read_excel,
    "xls": pd.read_excel,
    "xml": pd.read_xml,  # .. and so on
    "sql": my_read_sql,  # special case
}


def my_read_sql(con_w_sql, sep="#"):
    """Made-up function that reads SQL from CON in a single call

    Args:
        con_w_sql: connection string + query or table name,
            joined by separator string sep
        sep: separator

    Example:
        >>> my_read_sql('postgres:///db_name#table_name')
    """
    con, sql = special_path.split(sep)
    return pd.read_sql(sql, con)

In [ ]:
def read_any(file):
    _, ext = os.path.splitext(file)
    try:
        reader = READER_MAP[ext]
    except KeyError:
        raise ValueError(f"Unsupported filetype: {ext}")
    return reader(file)

In [ ]:
import pandas as pd
import os

file.endswith((".csv", ".tsv"))
# this is correct
file.endswith(".csv", ".tsv")
# this is wrong


file_path = input("enter the path to the file you want to open")
b = os.listdir(file_path)
for file in b:
    path = file_path + file
    df = pd.read_csv(path) if file.endswith((".csv", ".tsv")) else None
    df = pd.read_json(file) if file.endswith(".json") else df
    df = pd.read_xml(file) if file.endswith(".xml") else df
    df = pd.read_excel(file) if file.endswith((".xls", "xlsx")) else df
    df = pd.read_hdf(file) if file.endswith(".hdf") else df
    df = pd.read_sql(file) if file.endswith(".sql") else df
    if file.endswith(
        (".csv", ".tsv", ".json", ".xml", ".xls", ".xlxs", ".hdf", ".sql")
    ):
        print(df.head(100))
    else:
        print("format not supported")

In [ ]:
import os
import modin.pandas as pd
import json
import featuretools as ft
import matplotlib.pyplot as plt
import numpy as np
import time
from distributed import Client


def sqlread(con_w_sql, sep="#"):
    con, sql = special_path.split(sep)
    df = pd.read_sql(sql, con)
    return df.head()


readict = {
    ".csv": {"read": pd.read_csv},
    ".tsv": {"read": pd.read_csv},
    ".json": {"read": pd.read_json},
    ".xlsx": {"read": pd.read_excel},
    ".xml": {"read": pd.read_xml},
    ".xls": {"read": pd.read_excel},
    ".hdf": {"read": pd.read_hdf},
    ".sql": {"read": sqlread},
}


def read_any(file):
    _, ext = os.path.splitext(file)
    df = readict[ext]["read"](file)
    es = ft.EntitySet()
    es = es.entity_from_dataframe(entity_id="df", dataframe=df)
    head = df.head(100).values.tolist()
    dtype = es["df"]

    dtypedict = dtype.variable_types
    types = []

    for value in dtypedict.values():
        types.append(value.type_string)

    return json.dumps(head), json.dumps(types)


file = input("enter the path to the file you want to open : ")
read_any(file)